In [1]:
from __future__ import division
from __future__ import print_function
from sklearn import metrics
import random
import time
import sys
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import dgl
import dgl.function as fn
from dgl import DGLGraph
import numpy as np

from utils.utils import *
from models.gcn import GCN
from models.mlp import MLP

Using backend: pytorch


In [2]:
# Loading graph
adj, features,base_features, y_train, y_val, y_test, train_mask, val_mask, test_mask, train_size, test_size = load_corpus(word2vec=True)

features_with_word2vec = preprocess_features(features)

features = sp.identity(features.shape[0])
features = preprocess_features(features)

/home/bis/Projects/Classes/Deeplearning_class/project/utils/utils.py:242: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()


### Load image features

In [3]:
# Loading image features
train_embeddings, test_embeddings = get_image_embeddings()
training_embeddings = torch.tensor(train_embeddings).reshape(train_size,512)
test_embeddings = torch.tensor(test_embeddings).reshape(test_size,512)






/home/bis/Projects/venv3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  This is separate from the ipykernel package so we can avoid doing imports until


#### Load Wac

In [4]:
vocab_file = "./data/additional_data/meme_vocab.txt"
wac_data = loadWAC()


In [5]:
vocab = open(vocab_file, 'r')
all_word_embeddings = []
for word in vocab.readlines():
    word = word.strip()

    try:
        word_embedding = wac_data[word]
        word_embedding = torch.tensor(word_embedding).reshape(1,512)
    except:
        word_embedding = torch.zeros((1,512))
    all_word_embeddings.append(word_embedding)


In [6]:
image_embeddings_words  = torch.cat(all_word_embeddings, dim=0)

#### Combine wac and image representation

In [7]:
# Getting complete features for all nodes
word_nodes = features.shape[0] - train_size - test_size

# Since we don't have image embeddings for words, we will use zeros

all_image_features = torch.cat((training_embeddings, image_embeddings_words, test_embeddings), 0)

In [8]:
all_image_features.shape

torch.Size([20050, 512])

In [9]:
test_size

1000

In [10]:
image_embeddings_words

tensor([[-0.0091, -0.0294,  0.1800,  ...,  0.1068,  0.0904,  0.0022],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.2614, -0.0073, -0.1738,  ...,  0.4274,  0.2315,  0.0266],
        ...,
        [-0.0298,  0.0187, -0.1962,  ..., -0.1138,  0.0497,  0.0509],
        [ 0.0569,  0.1939, -0.0819,  ...,  0.2075, -0.2659, -0.0912],
        [-0.2407,  0.0551, -0.0462,  ...,  0.1517,  0.2010,  0.0911]])

In [11]:
def pre_adj(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj + sp.eye(adj.shape[0]))
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

adjdense = torch.from_numpy(pre_adj(adj).A.astype(np.float32))

In [12]:
CUDA = True
def construct_graph(adjacency):
    g = DGLGraph()
    adj = pre_adj(adjacency)
    g.add_nodes(adj.shape[0])
    g.add_edges(adj.row,adj.col)
    adjdense = adj.A
    adjd = np.ones((adj.shape[0]))
    for i in range(adj.shape[0]):
        adjd[i] = adjd[i] * np.sum(adjdense[i,:])
    weight = torch.from_numpy(adj.data.astype(np.float32))
    g.ndata['d'] = torch.from_numpy(adjd.astype(np.float32))
    g.edata['w'] = weight

    if CUDA:
        g = g.to(torch.device('cuda:0'))
    
    return g

In [13]:
class SimpleConv(nn.Module):
    def __init__(self,g,in_feats,out_feats,activation,feat_drop=True):
        super(SimpleConv, self).__init__()
        self.graph = g
        self.activation = activation
        #self.reset_parameters()
        setattr(self, 'W', nn.Parameter(torch.randn(in_feats,out_feats)))
        #self.b = nn.Parameter(torch.zeros(1, out_feats))
        #self.linear = nn.Linear(in_feats,out_feats)
        self.feat_drop = feat_drop
    
    # def reset_parameters(self):
    #     gain = nn.init.calculate_gain('relu')
    #     nn.init.xavier_uniform_(self.linear.weight,gain=gain)
    
    def forward(self, feat):
        g = self.graph.local_var()
        g.ndata['h'] = feat.mm(getattr(self, 'W'))
        g.update_all(fn.src_mul_edge(src='h', edge='w', out='m'), fn.sum(msg='m',out='h'))
        rst = g.ndata['h']
        #rst = self.linear(rst)
        rst = self.activation(rst)
        return rst

In [14]:
class GATLayer(nn.Module):
    def __init__(self, g, in_feats, out_feats):
        super(GATLayer, self).__init__()
        self.graph = g
        setattr(self, 'W', nn.Parameter(torch.randn(in_feats,out_feats)))
        setattr(self, 'al', nn.Parameter(torch.randn(in_feats,1)))
        setattr(self, 'ar', nn.Parameter(torch.randn(in_feats,1)))

    def forward(self, feat):
        # equation (1)
        g = self.graph.local_var()
        g.ndata['h'] = feat.mm(getattr(self, 'W'))
        g.ndata['el'] = feat.mm(getattr(self, 'al'))
        g.ndata['er'] = feat.mm(getattr(self, 'ar'))
        g.apply_edges(fn.u_add_v('el', 'er', 'e'))
        # message passing
        g.update_all(fn.src_mul_edge('h', 'w', 'm'), fn.sum('m', 'h'))
        e = F.leaky_relu(g.edata['e'])
        # compute softmax
        g.edata['w'] = F.softmax(e)
        rst = g.ndata['h']
        #rst = self.linear(rst)
        #rst = self.activation(rst)
        return rst

class MultiHeadGATLayer(nn.Module):
    def __init__(self, g, in_dim, out_dim, activation, num_heads=2, merge=None):
        super(MultiHeadGATLayer, self).__init__()
        self.heads = nn.ModuleList()
        for i in range(num_heads):
            self.heads.append(GATLayer(g, in_dim, out_dim))
        self.merge = merge
        self.activation = activation

    def forward(self, h):
        head_outs = [attn_head(h) for attn_head in self.heads]
        if self.merge == 'cat':
            # concat on the output feature dimension (dim=1)
            x = torch.cat(head_outs, dim=1)
        else:
            # merge using average
            x = torch.mean(torch.stack(head_outs),dim=0)
        
        return self.activation(x)

In [15]:
class SAGEMeanConv(nn.Module):
    def __init__(self,g,in_feats,out_feats,activation):
        super(SAGEMeanConv, self).__init__()
        self.graph = g
        self.feat_drop = nn.Dropout(0.5)
        setattr(self, 'W', nn.Parameter(torch.randn(in_feats,out_feats)))
        #self.linear = nn.Linear(in_feats, out_feats, bias=True)
        setattr(self, 'Wn', nn.Parameter(torch.randn(out_feats,out_feats)))
        self.activation = activation
        #self.neigh_linear = nn.Linear(out_feats, out_feats, bias=True)
        # self.reset_parameters()
    
    '''
    def reset_parameters(self):
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_uniform_(self.linear.weight,gain=gain)
        nn.init.xavier_uniform_(self.neigh_linear.weight,gain=gain)
    '''
    
    def forward(self,feat):
        g = self.graph.local_var()
        #feat = self.feat_drop(feat)
        h_self = feat.mm(getattr(self, 'W'))
        g.ndata['h'] = h_self
        g.update_all(fn.copy_src('h', 'm'), fn.sum('m', 'neigh'))
        h_neigh = g.ndata['neigh']
        degs = g.in_degrees().float()
        degs = degs.to(torch.device('cuda:0'))
        g.ndata['h'] = (h_neigh + h_self) / (degs.unsqueeze(-1) + 1)
        rst = g.ndata['h']
        rst = self.activation(rst)
        # rst = th.norm(rst)
        return rst

In [16]:
features_with_word2vec = torch.tensor(features_with_word2vec).to('cuda')

In [17]:

class Classifer(nn.Module):
    def __init__(self,g,input_dim,num_classes,conv):
        super(Classifer, self).__init__()
        self.data_graph = g
        self.GCN = conv
        self.gcn1 = self.GCN(g,input_dim, 300, F.relu)
        self.gcn2 = self.GCN(g, 300, 200, F.relu)
        self.gcn3 = self.GCN(g, 200, num_classes, F.relu)

    
    def forward(self, features):
        x = self.gcn1(features)

        # To Do: Fuse the text embedding with image embedding 
        self.embedding = x
        # x = torch.cat(x,g.ndata['x'])
        # x = torch.cat((self.embedding,g.ndata['x']),dim=1)
        x = self.gcn2(x)
        x = self.gcn3(x)
        
        return x




class ClassiferFusion(nn.Module):
    def __init__(self,g,input_dim,num_classes,conv):
        super(ClassiferFusion, self).__init__()
        self.data_graph = g
        self.GCN = conv
        self.gcn1 = self.GCN(g,input_dim, 300, F.relu)
        self.gcn2 = self.GCN(g, 300, 512, F.relu)
        self.gcn3 = self.GCN(g, 1024,num_classes , F.relu)
        # self.gcn4 = self.GCN(g, 300, num_classes, F.relu)
        # self.gcn5 = self.GCN(g, 300, num_classes, F.relu)


        self.dropout = nn.Dropout(0.1)


    
    def forward(self, features):
        x = self.gcn1(features)

        self.embedding1 = x
        # x = torch.cat(x,g.ndata['x'])
        # Mean of text embedding and word2vec
        x = torch.mean(torch.stack((self.embedding1,features_with_word2vec.float())),dim=0)

        # # Concatenate the text embedding with image embedding        

        x = self.gcn2(x)
        x = self.dropout(x)

        self.embedding2 = x
        x = torch.cat((self.embedding2,g.ndata['image_embeddings']),dim=1)

        x = self.gcn3(x)
        x = self.dropout(x)
        # x = self.gcn4(x)
        # x = self.dropout(x)
        # x = self.gcn5(x)
        
        return x

In [18]:
g = construct_graph(adj)


/home/bis/Projects/venv3.6/lib/python3.6/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


In [19]:
# Adding node features image_embeddings
all_image_features = all_image_features.cuda()
g.ndata['image_embeddings'] = all_image_features

In [20]:
# Initialize model
model = ClassiferFusion(g,input_dim=features.shape[0], num_classes=y_train.shape[1],conv=MultiHeadGATLayer)
# model = Classifer(g,input_dim=features.shape[0], num_classes=y_train.shape[1],conv=MultiHeadGATLayer)

In [21]:
# Define placeholders
t_features = torch.from_numpy(features.astype(np.float32))
t_y_train = torch.from_numpy(y_train)
t_y_val = torch.from_numpy(y_val)
t_y_test = torch.from_numpy(y_test)
t_train_mask = torch.from_numpy(train_mask.astype(np.float32))
tm_train_mask = torch.transpose(torch.unsqueeze(t_train_mask, 0), 1, 0).repeat(1, y_train.shape[1])
support = [preprocess_adj(adj)]
num_supports = 1
t_support = []
for i in range(len(support)):
    t_support.append(torch.Tensor(support[i]))

In [34]:
t_features = t_features.cuda()
t_y_train = t_y_train.cuda()
#t_y_val = t_y_val.cuda()
#t_y_test = t_y_test.cuda()
t_train_mask = t_train_mask.cuda()
tm_train_mask = tm_train_mask.cuda()
# for i in range(len(support)):
#     t_support = [t.cuda() for t in t_support if True]
model = model.cuda()

In [35]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01,weight_decay=0.01)

In [36]:
def evaluate(features, labels, mask):
    t_test = time.time()
    # feed_dict_val = construct_feed_dict(
    #     features, support, labels, mask, placeholders)
    # outs_val = sess.run([model.loss, model.accuracy, model.pred, model.labels], feed_dict=feed_dict_val)
    model.eval()
    with torch.no_grad():
        logits = model(features).cpu()
        t_mask = torch.from_numpy(np.array(mask*1., dtype=np.float32))
        tm_mask = torch.transpose(torch.unsqueeze(t_mask, 0), 1, 0).repeat(1, labels.shape[1])
        loss = criterion(logits * tm_mask, torch.max(labels, 1)[1])
        pred = torch.max(logits, 1)[1]
        acc = ((pred == torch.max(labels, 1)[1]).float() * t_mask).sum().item() / t_mask.sum().item()
        
    return loss.numpy(), acc, pred.numpy(), labels.numpy(), (time.time() - t_test)

val_losses = []

In [37]:
# Train model
epochs = 100
for epoch in range(epochs):

    t = time.time()
    
    # Forward pass
    logits = model(t_features)
    loss = criterion(logits * tm_train_mask, torch.max(t_y_train, 1)[1])    
    acc = ((torch.max(logits, 1)[1] == torch.max(t_y_train, 1)[1]).float() * t_train_mask).sum().item() / t_train_mask.sum().item()
        
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation
    val_loss, val_acc, pred, labels, duration = evaluate(t_features, t_y_val, val_mask)
    val_losses.append(val_loss)

    print_log("Epoch: {:.0f}, train_loss= {:.5f}, train_acc= {:.5f}, val_loss= {:.5f}, val_acc= {:.5f}, time= {:.5f}"\
                .format(epoch + 1, loss, acc, val_loss, val_acc, time.time() - t))

    # if epoch > 5 and val_losses[-1] > np.mean(val_losses[-(5+1):-1]):
    #     print_log("Early stopping...")
    #     break


print_log("Optimization Finished!")

/home/bis/Projects/venv3.6/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[2022/4/20 13:45:12] Epoch: 1, train_loss= 0.74704, train_acc= 0.60341, val_loss= 0.86229, val_acc= 0.52400, time= 1.21737
[2022/4/20 13:45:13] Epoch: 2, train_loss= 0.74464, train_acc= 0.60624, val_loss= 0.85394, val_acc= 0.52400, time= 1.24508
[2022/4/20 13:45:14] Epoch: 3, train_loss= 0.73731, train_acc= 0.61294, val_loss= 0.84681, val_acc= 0.52200, time= 1.23505
[2022/4/20 13:45:16] Epoch: 4, train_loss= 0.73251, train_acc= 0.61376, val_loss= 0.84131, val_acc= 0.52600, time= 1.20263
[2022/4/20 13:45:17] Epoch: 5, train_loss= 0.72986, train_acc= 0.61141, val_loss= 0.83678, val_acc= 0.52000, time= 1.22045
[2022/4/20 13:45:18] Epoch: 6, train_loss= 0.72787, train_acc= 0.61153, val_loss= 0.83270, val_acc= 0.51800, time= 1.23414
[2022/4/20 13:45:19] Epoch: 7, train_loss= 0.72584, train_acc= 0.61224, val_loss= 0.82887, val_acc= 0.52200, time= 1.21790
[2022/4/20 13:45:21] Epoch: 8, train_loss= 0.72373, train_acc= 0.61529, val_loss= 0.82514, val_acc= 0.52600, time= 1.21962
[2022/4/20 13:45

In [32]:
# Testing
test_loss, test_acc, pred, labels, test_duration = evaluate(t_features, t_y_test, test_mask)
print_log("Test set results: \n\t loss= {:.5f}, accuracy= {:.5f}, time= {:.5f}".format(test_loss, test_acc, test_duration))

test_pred = []
test_labels = []
for i in range(len(test_mask)):
    if test_mask[i]:
        test_pred.append(pred[i])
        test_labels.append(np.argmax(labels[i]))


print_log("Test Precision, Recall and F1-Score...")
print_log(metrics.classification_report(test_labels, test_pred, digits=4))
print_log("Macro average Test Precision, Recall and F1-Score...")
print_log(metrics.precision_recall_fscore_support(test_labels, test_pred, average='macro'))
print_log("Micro average Test Precision, Recall and F1-Score...")
print_log(metrics.precision_recall_fscore_support(test_labels, test_pred, average='micro'))

print_log("Auc Score test ...")
print_log(metrics.roc_auc_score(test_labels, test_pred, average='macro'))

/home/bis/Projects/venv3.6/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[2022/4/20 13:44:45] Test set results: 
[2022/4/20 13:44:45] 	 loss= 0.70657, accuracy= 0.51600, time= 0.48989
[2022/4/20 13:44:45] Test Precision, Recall and F1-Score...
[2022/4/20 13:44:45]               precision    recall  f1-score   support
[2022/4/20 13:44:45] 
[2022/4/20 13:44:45]            0     0.5159    0.8294    0.6361       510
[2022/4/20 13:44:45]            1     0.5167    0.1898    0.2776       490
[2022/4/20 13:44:45] 
[2022/4/20 13:44:45]     accuracy                         0.5160      1000
[2022/4/20 13:44:45]    macro avg     0.5163    0.5096    0.4569      1000
[2022/4/20 13:44:45] weighted avg     0.5163    0.5160    0.4604      1000
[2022/4/20 13:44:45] 
[2022/4/20 13:44:45] Macro average Test Precision, Recall and F1-Score...
[2022/4/20 13:44:45] (0.5162601626016261, 0.5096038415366146, 0.4568510829312087, None)
[2022/4/20 13:44:45] Micro average Test Precision, Recall and F1-Score...
[2022/4/20 13:44:45] (0.516, 0.516, 0.516, None)
[2022/4/20 13:44:45] Auc Sco